In [17]:
#installing library to read xlsx files
!pip install pandas openpyxl

In [18]:
from google.colab import files
uploaded = files.upload()

Saving FS - FacilityLocation.xlsx to FS - FacilityLocation (1).xlsx


In [19]:


# Import libraries
import openpyxl
import pandas as pd
import numpy as np  # Import NumPy for array manipulation


# Get the file name of the uploaded file
file_name = list(uploaded.keys())[0]

# Load the workbook
workbook = openpyxl.load_workbook(file_name, data_only=True)

# Initialize a dictionary to store DataFrames for each named range
named_ranges_data = {}

# Loop through all defined names in the workbook
for name, defined_name in workbook.defined_names.items():
    destinations = list(defined_name.destinations)
    if not destinations:
        continue  # Skip if there are no destinations

    for sheet_name, cell_range in destinations:
        try:
            sheet = workbook[sheet_name]

            # If it's a single cell, handle it differently
            if ":" not in cell_range:  # Single cell (no colon in the range)
                cell_value = sheet[cell_range].value
                # Convert the single cell value into a DataFrame (1x1)
                df = pd.DataFrame([[cell_value]])

            else:
                # Retrieve the data from the specified range
                data = []
                for row in sheet[cell_range]:
                    data.append([cell.value for cell in row])

                # Convert to DataFrame
                df = pd.DataFrame(data)

            # Store the DataFrame with the named range as the key
            named_ranges_data[name] = df

        except Exception as e:
            print(f"Error processing range {name}: {e}")
            continue

# Create an array for each DataFrame, named after the DataFrame
for range_name, df in named_ranges_data.items():
    globals()[range_name] = np.array(df)  # Converts the DataFrame to a NumPy array

# Example of accessing one of the dynamically created arrays
for range_name in named_ranges_data:
    print(f"Array created for {range_name}:")
    print(globals()[range_name])
    print()



Array created for budget:
[[5]]

Array created for factoryCost:
[[4]
 [2]]

Array created for factoryPresentValue:
[[4]
 [5]]

Array created for warehouseCost:
[[1]
 [2]]

Array created for warehousePresentValue:
[[2]
 [1]]



In [20]:
# Create a list factories
factories= list(range(0, factoryCost.shape[0]))
# Print the list 'sites' to verify
print("factories",factories)
# Create a list warehouses
warehouses = list(range(0, len(warehouseCost)))
# Print the list 'warehouses' to verify
print("warehouses:",warehouses)
# Assuming budget is a NumPy array, convert budget[0] to a scalar
budget_value = budget[0].item() if isinstance(budget[0], np.ndarray) else budget[0]
# Print the list 'budget' to verify
print("budget:",budget_value)

factories [0, 1]
warehouses: [0, 1]
budget: 5


In [21]:
#installing pyomo
!pip install -q pyomo

In [22]:
#installing coin or
!apt-get install -y -qq coinor-cbc

In [23]:
#importing pyomo environment
import pyomo.environ as pyo

In [24]:
#creating a model object
model = pyo.ConcreteModel()

In [25]:
#defining variables
model.x = pyo.Var(factories, within=pyo.Binary)
model.y = pyo.Var(warehouses, within=pyo.Binary)
model.x.pprint()
model.y.pprint()

x : Size=2, Index={0, 1}
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      0 :     0 :  None :     1 : False :  True : Binary
      1 :     0 :  None :     1 : False :  True : Binary
y : Size=2, Index={0, 1}
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      0 :     0 :  None :     1 : False :  True : Binary
      1 :     0 :  None :     1 : False :  True : Binary


In [26]:
# Remove any existing objective component named "obj"
if hasattr(model, 'obj'):
    model.del_component('obj')

#defining objective function
model.obj = pyo.Objective(
    expr = sum(factoryPresentValue[i]*model.x[i] for i in factories)+sum(warehousePresentValue[j]*model.y[j] for j in warehouses),
    sense = pyo.maximize )
model.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : maximize : 4*x[0] + 5*x[1] + 2*y[0] + y[1]


In [27]:
#modeling the coverage constraints
model.constraint_budget = pyo.ConstraintList()
model.constraint_budget.add(sum(factoryCost[i]*model.x[i] for i in factories)+sum(warehouseCost[j]*model.y[j] for j in warehouses) <=budget_value)
model.constraint_budget.pprint()

constraint_budget : Size=1, Index={1}, Active=True
    Key : Lower : Body                            : Upper : Active
      1 :  -Inf : 4*x[0] + 2*x[1] + y[0] + 2*y[1] :   5.0 :   True


In [28]:
#modeling the exclusive constraints
model.constraint_exclusive = pyo.ConstraintList()
model.constraint_exclusive.add(sum(model.y[i] for i in warehouses) <= 1)
model.constraint_exclusive.pprint()


constraint_exclusive : Size=1, Index={1}, Active=True
    Key : Lower : Body        : Upper : Active
      1 :  -Inf : y[0] + y[1] :   1.0 :   True


In [29]:
# Check if constraint_contingent already exists and remove it
if hasattr(model, 'constraint_contingent'):
    model.del_component(model.constraint_contingent)

#modeling the contingent constraints
model.constraint_contingent = pyo.ConstraintList()
for j in warehouses:
  model.constraint_contingent.add(model.y[j] <= model.x[j])
model.constraint_contingent.pprint()

constraint_contingent : Size=2, Index={1, 2}, Active=True
    Key : Lower : Body        : Upper : Active
      1 :  -Inf : y[0] - x[0] :   0.0 :   True
      2 :  -Inf : y[1] - x[1] :   0.0 :   True


In [30]:
#connecting cbc solver and printing the model
opt = pyo.SolverFactory('cbc')
model.pprint()

2 Var Declarations
    x : Size=2, Index={0, 1}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
    y : Size=2, Index={0, 1}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 4*x[0] + 5*x[1] + 2*y[0] + y[1]

3 Constraint Declarations
    constraint_budget : Size=1, Index={1}, Active=True
        Key : Lower : Body                            : Upper : Active
          1 :  -Inf : 4*x[0] + 2*x[1] + y[0] + 2*y[1] :   5.0 :   True
    constraint_contingent : Size=2, Index={1, 2}, Active=True
        Key : Lower : Body        : Upper : Active
          1 :  -Inf : y[0] - x[0] :   0

In [31]:
#solve the model
opt_solution = opt.solve(model)

In [32]:
# Print the values of the decision variables
#we are shifting the indices for presentation
print("\nFactories opening:")
for i in factories:
    if model.x[i].value > 0:
      print(f"{i+1}: {model.x[i].value}")

print("\nWarehouses opened:")
for j in warehouses:
    if model.y[j].value > 0:
      print(f"{j+1}: {model.y[j].value}")

print(f"Total net present value of the investment decision: {model.obj()}")


Factories opening:
2: 1.0

Warehouses opened:
2: 1.0
Total net present value of the investment decision: 6.0
